In [9]:
import os
from google.cloud import bigquery
import time

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'proud-climber-421817-14ec0fc568ac.json'
client = bigquery.Client()

In [2]:
# Construct table entity reference
dataset_ref = bigquery.DatasetReference(client.project, 'streamingDataSet')
table_ref = bigquery.TableReference(dataset_ref=dataset_ref, table_id='tbl_streamingData')
bqTable = client.get_table(table_ref)

In [3]:
# Get a snapshot of table schema
schemaOriginal = bqTable.schema

print(schemaOriginal)

[SchemaField('data', 'STRING', 'NULLABLE', None, None, (), None)]


In [4]:
# Add New Columns
schemaNew = schemaOriginal[:] # Create a copy of original schema

schemaNew.append(bigquery.SchemaField("newCol1", "BOOLEAN", mode="NULLABLE"))
schemaNew.append(bigquery.SchemaField("newCol2", "STRING", mode="NULLABLE"))

print(schemaNew)

[SchemaField('data', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('newCol1', 'BOOLEAN', 'NULLABLE', None, None, (), None), SchemaField('newCol2', 'STRING', 'NULLABLE', None, None, (), None)]


In [5]:
# Assign the updated schema to the BQ Table
bqTable.schema = schemaNew

# Make an API rquest to add Column
client.update_table(bqTable, ['schema'])

Table(TableReference(DatasetReference('proud-climber-421817', 'streamingDataSet'), 'tbl_streamingData'))

In [7]:
# Delete Columns
query_job = client.query(
    """
    ALTER TABLE streamingDataSet.tbl_streamingData
    DROP COLUMN IF EXISTS newCol1,
    DROP COLUMN IF EXISTS newCol2
    """
)

In [10]:
while query_job.state != 'DONE':
    print("Waiting for the job to finish....")
    time.sleep(3)
    query_job.reload()

print(query_job.result())

Waiting for the job to finish....
